In [69]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Photon
from geopy.extra.rate_limiter import RateLimiter
import re

In [90]:
df = pd.read_csv('mt20_tcoi.csv', sep=';', encoding='latin1', usecols=["TYPE", "DATE", "CIBLE", "CORRESPONDANT", "DIRECTION", "DUREE", "IMSI", "IMEI", "ADRESSE2", "CODE POSTAL", "VILLE", "X", "Y"], dtype= {"TYPE": str, "CIBLE": str, "CORRESPONDANT": str, "DIRECTION": str, "DUREE": str, "IMSI": str, "IMEI": str, "ADRESSE2": str, "CODE POSTAL": str, "VILLE": str, "X": str, "Y": str})
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135


In [91]:
# Fonction pour convertir les dates
def convert_date(date_str):
    # Extraire le fuseau horaire avec une expression régulière
    match = re.search(r'UTC([+-]\d)', date_str)
    if match:
        utc_offset = int(match.group(1))  # Récupérer le décalage horaire
    else:
        utc_offset = 0  # Valeur par défaut si aucun fuseau horaire n'est trouvé

    # Retirer le fuseau horaire et convertir en datetime
    date_without_tz = date_str.split(' UTC')[0]  # Retirer ' UTC+X'
    dt = pd.to_datetime(date_without_tz, format='%d/%m/%Y - %H:%M:%S')

    # Ajuster le fuseau horaire à UTC+4
    dt = dt + pd.Timedelta(hours=(4 - utc_offset))  # Ajustement basé sur l'offset

    return dt

# Appliquer la fonction pour convertir les dates
df['converted_date'] = df['DATE'].apply(convert_date)

# Afficher le DataFrame avec les dates converties
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 04:29:52


In [92]:
# Formater la date au format DD-MM-YYYY HH:MM:SS
df['formatted_date'] = df['converted_date'].dt.strftime('%d-%m-%Y %H:%M:%S')

In [93]:
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,"=(""262693878601"")",NaN,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52


In [94]:
df["CORRESPONDANT"] = df["CORRESPONDANT"].fillna("Data")
df["DUREE"] = df["DUREE"].fillna("0")
df['IMEI'] = df['IMEI'].fillna("Non précisé")


In [95]:
print(df.isna().sum())

TYPE                 0
DATE                 0
CIBLE                0
CORRESPONDANT        0
DIRECTION            0
DUREE                0
IMSI                 0
IMEI                 0
ADRESSE2          1052
CODE POSTAL       1052
VILLE             1052
X                 1052
Y                 1052
converted_date       0
formatted_date       0
dtype: int64


In [96]:
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,"=(""262693878601"")",Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,"=(""262693878601"")",Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,"=(""262693878601"")",Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,"=(""262693878601"")",Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,"=(""262693878601"")",Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52


In [97]:
# Fonction pour nettoyer les chaînes
def clean_number(number_str):
    # Utiliser une expression régulière pour supprimer le préfixe et le suffixe
    return re.sub(r'^\=\("\s*|\s*"\)$', '', number_str)

In [98]:
df['CORRESPONDANT'] = df['CORRESPONDANT'].apply(clean_number)
df['CIBLE'] = df['CIBLE'].apply(clean_number)
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,262693878601,Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,262693878601,Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,262693878601,Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,262693878601,Data,sortant,3600,"=(""647030000326990"")","=(""356934103953870"")",Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52


In [99]:
df["IMEI"] = df["IMEI"].apply(clean_number)
df['IMSI'] = df['IMSI'].apply(clean_number)
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,ville inconnue,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52


In [100]:
reunion_postal_codes = {
    "97400": "Saint-Denis",
    "97410": "Saint-Pierre",
    "97411": "Bois de Nèfles",
    "97412": "Bras Panon",
    "97413": "Cilaos",
    "97414": "Entre Deux",
    "97416": "La Chaloupe",
    "97417": "Saint-Bernard",
    "97418": "La Plaine des Cafres",
    "97419": "La Possession",
    "97420": "Le Port",
    "97421": "La Rivière Saint-Louis",
    "97422": "La Saline",
    "97423": "Le Guillaume",
    "97424": "Piton Saint-Leu",
    "97425": "Les Avirons",
    "97426": "Les Trois Bassins",
    "97427": "L'Étang-Salé",
    "97428": "La Nouvelle",
    "97429": "Petite-Île",
    "97430": "Le Tampon",
    "97431": "La Plaine des Palmistes",
    "97432": "La Ravine des Cabris",
    "97433": "Salazie - Hell-bourg",
    "97434": "Saint-Gilles-les-Bains",
    "97435": "Bernica",
    "97436": "Saint-Leu",
    "97437": "Sainte-Anne",
    "97438": "Sainte-Marie",
    "97439": "Sainte-Rose",
    "97440": "Saint-André",
    "97441": "Sainte-Suzanne",
    "97442": "Basse Vallée",
    "97450": "Saint-Louis",
    "97460": "Saint-Paul",
    "97470": "Saint-Benoît",
}

In [101]:
def replace_unknown_ville(row):
    if row['VILLE'] == 'ville inconnue':
        return reunion_postal_codes.get(row['CODE POSTAL'], 'ville inconnue')  # Remplace par la ville correspondante ou garde 'ville inconnue'
    return row['VILLE']

# Appliquer la fonction à la colonne 'ville'
df['VILLE'] = df.apply(replace_unknown_ville, axis=1)
df['VILLE']= df['VILLE'].str.replace("-", " ")
df['VILLE']= df['VILLE'].str.replace("SAINT", "ST")
df['VILLE']= df['VILLE'].str.replace("SAINTE", "STE")
df['VILLE']= df['VILLE'].str.replace("L'", "")
df['VILLE'] = df['VILLE'].str.upper()

# Afficher le DataFrame avec les villes mises à jour
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52


In [102]:
df["Adresse"] = df["ADRESSE2"] + ", " + df["CODE POSTAL"] + " " + df["VILLE"]
df['Adresse'] = df['Adresse'].str.upper()
df.head()

,TYPE,DATE,CIBLE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,ADRESSE2,CODE POSTAL,VILLE,X,Y,converted_date,formatted_date,Adresse
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"
2,DATA_PAQUET,01/09/2024 - 00:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 02:29:52,01-09-2024 02:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"
3,DATA_PAQUET,01/09/2024 - 01:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 03:29:52,01-09-2024 03:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"
4,DATA_PAQUET,01/09/2024 - 02:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 04:29:52,01-09-2024 04:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"


In [103]:
df = df.rename(columns={"TYPE": "Type d'appel", "CORRESPONDANT": "Correspondant", "CIBLE": "Abonné", "DIRECTION": "Direction", "DUREE": "Durée", "VILLE": "Ville", "X": "Latitude", "Y": "Longitude", "formatted_date": "Date"})

In [104]:
df.head(2)

,Type d'appel,DATE,Abonné,Correspondant,Direction,Durée,IMSI,IMEI,ADRESSE2,CODE POSTAL,Ville,Latitude,Longitude,converted_date,Date,Adresse
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"
1,DATA_PAQUET,31/08/2024 - 23:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 01:29:52,01-09-2024 01:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS"


In [105]:
# Extraire l'année, le mois et le jour de la semaine
df['Années'] = df['converted_date'].dt.year
df['Mois'] = df['converted_date'].dt.month
df['Jour de la semaine'] = df['converted_date'].dt.day_name()

# Mapper les jours de la semaine en français
jours_semaine_fr = {
    'Monday': 'Lundi',
    'Tuesday': 'Mardi',
    'Wednesday': 'Mercredi',
    'Thursday': 'Jeudi',
    'Friday': 'Vendredi',
    'Saturday': 'Samedi',
    'Sunday': 'Dimanche'
}

# Mapper les mois en français
mois_fr = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}

# Remplacer les numéros de mois par leur équivalent en français
df['Mois'] = df['Mois'].map(mois_fr)
# Remplacer les noms des jours par leur équivalent en français
df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)

In [106]:
df.head(1)

,Type d'appel,DATE,Abonné,Correspondant,Direction,Durée,IMSI,IMEI,ADRESSE2,CODE POSTAL,Ville,Latitude,Longitude,converted_date,Date,Adresse,Années,Mois,Jour de la semaine
0,DATA_PAQUET,31/08/2024 - 22:29:52 UTC+2,262693878601,Data,sortant,3600,647030000326990,356934103953870,Chemin Canot,97421,LA RIVIÈRE SAINT LOUIS,-21.243958,55.421135,2024-09-01 00:29:52,01-09-2024 00:29:52,"CHEMIN CANOT, 97421 LA RIVIÈRE SAINT LOUIS",2024,Septembre,Dimanche


In [107]:
address_df = df[['Adresse', 'Longitude', 'Latitude']]

# Compter le nombre d'adresses groupées
address_count = address_df.groupby(['Adresse', 'Longitude', 'Latitude']).size().reset_index(name='Count')

# Calculer le total des occurrences
total_count = address_count['Count'].sum()

# Calculer le pourcentage pour chaque adresse
address_count['Percentage'] = (address_count['Count'] / total_count) * 100

address_count = address_count.dropna(axis=0)

address_count['Longitude'] = address_count['Longitude'].astype(float)

address_count['Latitude'] = address_count['Latitude'].astype(float)

address_count = address_count.dropna(axis=0)

address_count.head()

,Adresse,Longitude,Latitude,Count,Percentage
0,"1, RUE FLEUR JAUNE LA FENETRE, 97421 ST LOUIS",55.433600,-21.185600,3,0.118483
1,"25A ALLEE CANNES BONBON, 97450 SAINT LOUIS",55.435055,-21.285473,87,3.436019
2,"BATIMENT A LA CHAINE - RUE DES BONS ENFANTS, ...",55.466600,-21.336000,8,0.315956
3,"CENTRE COMMERCIAL DU PORTAIL, 97436 ST LEU",55.300600,-21.218200,2,0.078989
4,"CHEMIN BELLECOMBE, 97425 LES AVIRONS",55.336300,-21.232300,1,0.039494


In [110]:
fig = px.scatter_map(address_count, lat="Latitude", lon="Longitude", color="Percentage", size="Count", hover_name="Adresse",
                    color_continuous_scale=px.colors.sequential.Bluered, size_max=15, zoom=10,
                    map_style="carto-positron")

In [111]:
fig.show()

In [112]:
adresse_counts = df['Adresse'].value_counts().reset_index()

adresse_counts.head()

,Adresse,count
0,"142 RUE LECONTE DELISLE, 97450 SAINT LOUIS",1611
1,"LE RUISSEAU TERRE ROUGE, 97421 ST LOUIS",245
2,"RUE DES SALINS, 97460 ST PAUL",195
3,"50 RUE HENRI LAPIERRE RES. GOYAVES MARRONS, 97...",189
4,"25A ALLEE CANNES BONBON, 97450 SAINT LOUIS",87
